In [3]:
import pandas as pd
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime,date
import src.info as info
import src.in_out as io
import numpy as np
import itertools
import src.graphic as graph
from glob import glob

In [253]:
#Veriler, tarih ve ürün adları ile okunuyor ve birleştiriliyor
path_list=io.get_path('data')
cols= ["Time","BID price","ASK price"]
master_data=pd.concat((pd.read_excel(file,usecols=cols).\
                       assign(name=info.get_productName(file),date=info.get_productDate(file)
                             ) for file in path_list)).reset_index(drop=True)
master_data.columns=['time','bid_price','ask_price','name','date']

In [258]:
#date sütunu Timestamp'e dönüştürülüyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')

In [254]:
#tarih ve saat bilgisi birleştiriliyor
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [284]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

In [256]:
# mid price hesaplanıyor
master_data['mid_price']=aux.get_mid_price(master_data)

In [305]:
group=master_data.groupby(master_data.date.dt.date)

In [307]:
master_data

,bid_price,ask_price,name,date,mid_price,time_period
0,0.668949,0.7193,6AU8,2018-09-06 18:00:00,0.694125,18
1,0.668856,0.7194,6AU8,2018-09-06 18:00:01,0.694128,18
2,0.668856,0.7193,6AU8,2018-09-06 18:00:02,0.694078,18
3,0.668856,0.7193,6AU8,2018-09-06 18:00:03,0.694078,18
4,0.668856,0.7193,6AU8,2018-09-06 18:00:04,0.694078,18
...,...,...,...,...,...,...
237595,0.719500,0.7196,6CU8,2018-09-07 04:59:55,0.719550,4
237596,0.719500,0.7196,6CU8,2018-09-07 04:59:56,0.719550,4
237597,0.719500,0.7196,6CU8,2018-09-07 04:59:57,0.719550,4
237598,0.719500,0.7196,6CU8,2018-09-07 04:59:58,0.719550,4
